In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split

import torch
import torchvision
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from PIL import Image
from torchvision import transforms

import timm
import torch.nn as nn
from torchinfo import summary
from tqdm import tqdm

import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

import gc

In [2]:
batch_size = 16
os.makedirs('outputs', exist_ok=True)
os.makedirs('outputs/run', exist_ok=True)

# Dataset

## Load

In [3]:
def loading_the_data(data_dir):
    # Generate data paths with labels
    filepaths = []
    labels = []

    # Get folder names
    folds = os.listdir(data_dir)

    for fold in folds:
        foldpath = os.path.join(data_dir, fold)
        filelist = os.listdir(foldpath)
        for file in filelist:
            fpath = os.path.join(foldpath, file)
            
            filepaths.append(fpath)
            labels.append(fold)

    # Concatenate data paths with labels into one DataFrame
    Fseries = pd.Series(filepaths, name='filepaths')
    Lseries = pd.Series(labels, name='labels')

    df = pd.concat([Fseries, Lseries], axis=1)
    
    return df

In [4]:
dir = 'G:\\FireRisk'

train_df = loading_the_data(dir + '\\train')
test_df = loading_the_data(dir + '\\val')

train_df

,filepaths,labels
0,G:\FireRisk\train\High\27032281_4_-103.4304412...,High
1,G:\FireRisk\train\High\27038991_4_-77.77273442...,High
2,G:\FireRisk\train\High\27040201_4_-73.83896834...,High
3,G:\FireRisk\train\High\27042071_4_-122.1662712...,High
4,G:\FireRisk\train\High\27042401_4_-121.1231610...,High
...,...,...
70326,G:\FireRisk\train\Water\35471591_7_-72.4088150...,Water
70327,G:\FireRisk\train\Water\35484351_7_-82.8478475...,Water
70328,G:\FireRisk\train\Water\35487101_7_-72.1588909...,Water
70329,G:\FireRisk\train\Water\35497331_7_-90.9452064...,Water


## Menambahkan Label Kontinu

In [5]:
cnt_df = pd.read_csv("conversion_cnt.csv")
train_df = pd.merge(train_df, cnt_df[['filepaths', 'grid_code']], on='filepaths', how='left')
train_df.rename(columns={'grid_code': 'labels_cnt'}, inplace=True)

cnt_df = pd.read_csv("conversion_test_cnt.csv")
test_df = pd.merge(test_df, cnt_df[['filepaths', 'grid_code']], on='filepaths', how='left')
test_df.rename(columns={'grid_code': 'labels_cnt'}, inplace=True)

train_df

,filepaths,labels,labels_cnt
0,G:\FireRisk\train\High\27032281_4_-103.4304412...,High,1237
1,G:\FireRisk\train\High\27038991_4_-77.77273442...,High,628
2,G:\FireRisk\train\High\27040201_4_-73.83896834...,High,718
3,G:\FireRisk\train\High\27042071_4_-122.1662712...,High,805
4,G:\FireRisk\train\High\27042401_4_-121.1231610...,High,1093
...,...,...,...
70326,G:\FireRisk\train\Water\35471591_7_-72.4088150...,Water,0
70327,G:\FireRisk\train\Water\35484351_7_-82.8478475...,Water,0
70328,G:\FireRisk\train\Water\35487101_7_-72.1588909...,Water,0
70329,G:\FireRisk\train\Water\35497331_7_-90.9452064...,Water,0


## Encoding Label Kelas

In [6]:
class_names = ['Water', 'Non-burnable', 'Very_Low', 'Low', 'Moderate', 'High', 'Very_High']
label_encoder = OrdinalEncoder(categories=[class_names])

train_df['labels'] = label_encoder.fit_transform(train_df[['labels']])
test_df['labels'] = label_encoder.transform(test_df[['labels']])

train_df['labels'] = train_df['labels'].astype('int64')
test_df['labels'] = test_df['labels'].astype('int64')

train_df

,filepaths,labels,labels_cnt
0,G:\FireRisk\train\High\27032281_4_-103.4304412...,5,1237
1,G:\FireRisk\train\High\27038991_4_-77.77273442...,5,628
2,G:\FireRisk\train\High\27040201_4_-73.83896834...,5,718
3,G:\FireRisk\train\High\27042071_4_-122.1662712...,5,805
4,G:\FireRisk\train\High\27042401_4_-121.1231610...,5,1093
...,...,...,...
70326,G:\FireRisk\train\Water\35471591_7_-72.4088150...,0,0
70327,G:\FireRisk\train\Water\35484351_7_-82.8478475...,0,0
70328,G:\FireRisk\train\Water\35487101_7_-72.1588909...,0,0
70329,G:\FireRisk\train\Water\35497331_7_-90.9452064...,0,0


## Normalisasi Label Kontinu

In [7]:
def normalize_cont_label(label):
    if label <= 0:
        return 0
    elif label <= 61:
        return label / 61
    elif 61 < label <= 178:
        return (label - 61) / (178 - 61) + 1
    elif 178 < label <= 489:
        return (label - 178) / (489 - 178) + 2
    elif 489 < label <= 1985:
        return (label - 489) / (1985 - 489) + 3
    elif 1985 < label:
        return (label - 1985) / (100000 - 1985) + 4

In [8]:
train_df['labels_cnt'] = train_df['labels_cnt'].apply(normalize_cont_label).round(3)
test_df['labels_cnt'] = test_df['labels_cnt'].apply(normalize_cont_label).round(3)

train_df

,filepaths,labels,labels_cnt
0,G:\FireRisk\train\High\27032281_4_-103.4304412...,5,3.500
1,G:\FireRisk\train\High\27038991_4_-77.77273442...,5,3.093
2,G:\FireRisk\train\High\27040201_4_-73.83896834...,5,3.153
3,G:\FireRisk\train\High\27042071_4_-122.1662712...,5,3.211
4,G:\FireRisk\train\High\27042401_4_-121.1231610...,5,3.404
...,...,...,...
70326,G:\FireRisk\train\Water\35471591_7_-72.4088150...,0,0.000
70327,G:\FireRisk\train\Water\35484351_7_-82.8478475...,0,0.000
70328,G:\FireRisk\train\Water\35487101_7_-72.1588909...,0,0.000
70329,G:\FireRisk\train\Water\35497331_7_-90.9452064...,0,0.000


In [9]:
train_df.describe()

,labels,labels_cnt
count,70331.000000,70331.000000
mean,2.547156,1.109816
std,1.498586,1.271991
min,0.000000,0.000000
25%,1.000000,0.000000
50%,2.000000,0.557000
75%,4.000000,2.035000
max,6.000000,4.576000


## Split Data

In [10]:
train_df, valid_df = train_test_split(train_df, test_size = 0.2, shuffle = True, random_state = 49, stratify=train_df['labels'])

## Augmentasi

In [11]:
class FireRiskDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        # Return the number of samples in the dataset
        return len(self.dataframe)

    def __getitem__(self, idx):
        # Get the file path and label for the index
        img_path = self.dataframe.iloc[idx, 0]
        label = self.dataframe.iloc[idx, 1]
        label_cnt = self.dataframe.iloc[idx, 2]
        
        # Open the image
        image = Image.open(img_path).convert("RGB")

        # If there is any transform (e.g., normalization, augmentation), apply it
        if self.transform:
            image = self.transform(image)

        return image, label, label_cnt, img_path

In [12]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224
    # transforms.CenterCrop(224),  # Crop image to get 224x224 in the center
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize with ImageNet stats
])

augment = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224
    # transforms.RandomHorizontalFlip(p=0.2),  # Randomly flip image horizontally
    # transforms.RandomAffine(degrees=10, translate=(0.03125, 0.03125), fill=(0, 0, 0)),  # Random affine transformations (rotation, translation)
    # transforms.CenterCrop(224),  # Crop image to get 224x224 in the center
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize with ImageNet stats
])

# Create the dataset
train_dataset = FireRiskDataset(dataframe=train_df, transform=augment)
valid_dataset = FireRiskDataset(dataframe=valid_df, transform=transform)
test_dataset = FireRiskDataset(dataframe=test_df, transform=transform)

# Create a DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Model

In [13]:
if 'model' in globals() and model != None:
    model.cpu()
    del model
if 'mae_model' in globals() and mae_model != None:
    mae_model.cpu()
    del mae_model
if 'full_model' in globals() and full_model != None:
    full_model.cpu()
    del full_model
torch.cuda.empty_cache()
gc.collect()

36

In [14]:
# Use GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


## MAE Model

In [15]:
# Load MAE model
mae_model = timm.create_model('vit_base_patch16_224', pretrained=False)

# Load the pre-trained weights
checkpoint = torch.load('mae_pretrain_vit_base.pth', weights_only=True)
state_dict = checkpoint['model'] if 'model' in checkpoint else checkpoint
mae_model.load_state_dict(state_dict, strict=False)

# Remove the final classification head (to use only the encoder part)
mae_model.reset_classifier(0)
mae_model = mae_model.to(device)

In [16]:
# Freeze all parameters in the encoder
for param in mae_model.parameters():
    param.requires_grad = False

In [17]:
summary(mae_model, input_size=(batch_size, 3, 224, 224))

Layer (type:depth-idx)                   Output Shape              Param #
VisionTransformer                        [16, 768]                 152,064
├─PatchEmbed: 1-1                        [16, 196, 768]            --
│    └─Conv2d: 2-1                       [16, 768, 14, 14]         (590,592)
│    └─Identity: 2-2                     [16, 196, 768]            --
├─Dropout: 1-2                           [16, 197, 768]            --
├─Identity: 1-3                          [16, 197, 768]            --
├─Identity: 1-4                          [16, 197, 768]            --
├─Sequential: 1-5                        [16, 197, 768]            --
│    └─Block: 2-3                        [16, 197, 768]            --
│    │    └─LayerNorm: 3-1               [16, 197, 768]            (1,536)
│    │    └─Attention: 3-2               [16, 197, 768]            (2,362,368)
│    │    └─Identity: 3-3                [16, 197, 768]            --
│    │    └─Identity: 3-4                [16, 197, 768]    

### Latent Extraction

In [18]:
def extract_latent_representations(dataloader, model, device):
    model.eval()
    latent_representations = []
    labels = []
    labels_cnt = []
    filenames = []

    with torch.no_grad():
        for images, targets, targets_cnt, filename in tqdm(dataloader, unit="batch"):
            images = images.to(device)

            # Forward pass through the MAE encoder
            latent = model(images)

            latent_representations.append(latent.cpu())
            labels.extend(targets)
            labels_cnt.extend(targets_cnt)
            filenames.extend(filename)

    # Concatenate the results across batches
    latent_representations = torch.cat(latent_representations, dim=0)

    return latent_representations, labels, labels_cnt, filenames

## Head Model

In [21]:
class FireRisk_Head(nn.Module):
    def __init__(self, num_classes=7, dropout_prob=0.5, latent_dim=768):
        super(FireRisk_Head, self).__init__()

        input_dim = latent_dim
        
        # Shared layers
        self.shared = nn.Module()
        
        # From latent representation to 512 neurons
        self.shared.fc1 = nn.Linear(768, 512)
        self.shared.bn1 = nn.BatchNorm1d(512)
        self.shared.dropout1 = nn.Dropout(dropout_prob)
        
        # # From 512 neurons to 256 neurons
        # self.shared.fc2 = nn.Linear(512, 256)
        # self.shared.bn2 = nn.BatchNorm1d(256)
        # self.shared.dropout2 = nn.Dropout(dropout_prob)
        
        # Classification module
        self.classification = nn.Module()
        self.classification.fc1 = nn.Linear(512, 128)
        self.classification.bn1 = nn.BatchNorm1d(128)
        self.classification.dropout1 = nn.Dropout(dropout_prob)
        self.classification.head = nn.Linear(128, num_classes)

        # Regression module
        self.regression = nn.Module()
        self.regression.fc1 = nn.Linear(512, 128)
        self.regression.bn1 = nn.BatchNorm1d(128)
        self.regression.dropout1 = nn.Dropout(dropout_prob)
        self.regression.head = nn.Linear(128, 1)

    def forward(self, x):
        # Fully connected layer (512 neurons)
        x = self.shared.fc1(x)
        x = self.shared.bn1(x)
        x = torch.relu(x)
        x = self.shared.dropout1(x)

        # # Fully connected layer (256 neurons)
        # x = self.shared.fc2(x)
        # x = self.shared.bn2(x)
        # x = torch.relu(x)
        # x = self.shared.dropout2(x)

        # Classification head (7 classes)
        cls = self.classification.fc1(x)
        cls = self.classification.bn1(cls)
        cls = torch.relu(cls)
        cls = self.classification.dropout1(cls)
        cls = self.classification.head(cls)

        # Regression head (1 continuous)
        reg = self.regression.fc1(x)
        reg = self.regression.bn1(reg)
        reg = torch.relu(reg)
        reg = self.regression.dropout1(reg)
        reg = self.regression.head(reg)
        
        return cls, reg

In [22]:
# Initialize the model
model = FireRisk_Head(num_classes=7)
summary(model, input_size=(batch_size, 768))

Layer (type:depth-idx)                   Output Shape              Param #
FireRisk_Head                            [16, 7]                   --
├─Module: 1-1                            --                        --
│    └─Linear: 2-1                       [16, 512]                 393,728
│    └─BatchNorm1d: 2-2                  [16, 512]                 1,024
│    └─Dropout: 2-3                      [16, 512]                 --
├─Module: 1-2                            --                        --
│    └─Linear: 2-4                       [16, 128]                 65,664
│    └─BatchNorm1d: 2-5                  [16, 128]                 256
│    └─Dropout: 2-6                      [16, 128]                 --
│    └─Linear: 2-7                       [16, 7]                   903
├─Module: 1-3                            --                        --
│    └─Linear: 2-8                       [16, 128]                 65,664
│    └─BatchNorm1d: 2-9                  [16, 128]                 

## Full Model

In [23]:
class FireRisk_Full(nn.Module):
    def __init__(self, mae_model, head_model, num_classes=7, dropout_prob=0.5):
        super(FireRisk_Full, self).__init__()
        
        # MAE encoder
        self.mae_encoder = mae_model
        
        # Prediction head
        self.head = head_model

    def forward(self, x):
        # Pass through the MAE encoder to get the latent representation
        x = self.mae_encoder(x)

        # Prediction head
        cls, reg = self.head(x)
        
        return cls, reg

In [24]:
# Initialize the model
full_model = FireRisk_Full(mae_model=mae_model, head_model=model, num_classes=7)
summary(full_model, input_size=(batch_size, 3, 224, 224))

Layer (type:depth-idx)                        Output Shape              Param #
FireRisk_Full                                 [16, 7]                   --
├─VisionTransformer: 1-1                      [16, 768]                 152,064
│    └─PatchEmbed: 2-1                        [16, 196, 768]            --
│    │    └─Conv2d: 3-1                       [16, 768, 14, 14]         (590,592)
│    │    └─Identity: 3-2                     [16, 196, 768]            --
│    └─Dropout: 2-2                           [16, 197, 768]            --
│    └─Identity: 2-3                          [16, 197, 768]            --
│    └─Identity: 2-4                          [16, 197, 768]            --
│    └─Sequential: 2-5                        [16, 197, 768]            --
│    │    └─Block: 3-3                        [16, 197, 768]            (7,087,872)
│    │    └─Block: 3-4                        [16, 197, 768]            (7,087,872)
│    │    └─Block: 3-5                        [16, 197, 768]     

In [25]:
if 'model' in globals() and model != None:
    model.cpu()
    del model
if 'full_model' in globals() and full_model != None:
    full_model.cpu()
    del full_model
torch.cuda.empty_cache()
gc.collect()

504

# Train

In [26]:
class LatentDataset(Dataset):
    def __init__(self, latents, labels, labels_cnt, filenames):
        self.latents = latents
        # self.glcm = glcm
        # self.lbp = lbp
        self.labels = labels
        self.labels_cnt = labels_cnt
        self.filenames = filenames

    def __len__(self):
        # Return the number of samples in the dataset
        return len(self.latents)

    def __getitem__(self, idx):
        # Get the latent and label for the index
        latent = self.latents[idx]
        # glcm_feat = self.glcm[idx]
        # lbp_feat = self.lbp[idx]
        label = self.labels[idx]
        label_cnt = self.labels_cnt[idx]
        filename = self.filenames[idx]

        # combined = torch.cat([latent, glcm_feat, lbp_feat], dim=0).float()

        return latent, label, label_cnt, filename

### Load latents

In [143]:
# Load precomputed latent representations and labels
train_data = torch.load('outputs/train_latents.pth', weights_only=True)
valid_data = torch.load('outputs/valid_latents.pth', weights_only=True)

# train_latents, train_glcm, train_lbp, train_labels, train_labels_cnt, train_filenames = train_data['latents'], train_data['glcm'], train_data['lbp'], train_data['labels'], train_data['labels_cnt'], train_data['filenames']
# valid_latents, valid_glcm, valid_lbp, valid_labels, valid_labels_cnt, valid_filenames = valid_data['latents'], valid_data['glcm'], valid_data['lbp'], valid_data['labels'], valid_data['labels_cnt'], valid_data['filenames']
train_latents, train_labels, train_labels_cnt, train_filenames = train_data['latents'], train_data['labels'], train_data['labels_cnt'], train_data['filenames']
valid_latents, valid_labels, valid_labels_cnt, valid_filenames = valid_data['latents'], valid_data['labels'], valid_data['labels_cnt'], valid_data['filenames']

# Min-max normalization
min_val = np.min(train_labels_cnt)
max_val = np.max(train_labels_cnt)
train_labels_cnt_norm = (train_labels_cnt - min_val) / (max_val - min_val)
valid_labels_cnt_norm = (valid_labels_cnt - min_val) / (max_val - min_val)

# Create DataLoaders using the precomputed latents
# train_lat_dataset = LatentDataset(train_latents[0], train_glcm[0], train_lbp[0], train_labels, train_labels_cnt_norm, train_filenames)
# valid_lat_dataset = LatentDataset(valid_latents, valid_glcm, valid_lbp, valid_labels, valid_labels_cnt_norm, valid_filenames)
train_lat_dataset = LatentDataset(train_latents, train_labels, train_labels_cnt_norm, train_filenames)
valid_lat_dataset = LatentDataset(valid_latents, valid_labels, valid_labels_cnt_norm, valid_filenames)

train_lat_loader = DataLoader(train_lat_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
valid_lat_loader = DataLoader(valid_lat_dataset, batch_size=batch_size, shuffle=False)

### Class imbalance

In [28]:
classes, counts = np.unique(train_labels, return_counts=True)
class_weights = {c: 1.0 / count for c, count in zip(classes, counts)}

mean = sum(class_weights.values()) / len(class_weights)
class_weights = {k: v / mean for k, v in class_weights.items()}

weights_list = [class_weights[i] for i in range(len(classes))]
weights_tensor = torch.tensor(weights_list, dtype=torch.float).to(device)

class_weights

{np.int64(0): np.float64(2.9870853377876045),
 np.int64(1): np.float64(0.2879995261284951),
 np.int64(2): np.float64(0.23769682234046724),
 np.int64(3): np.float64(0.48302547493704984),
 np.int64(4): np.float64(0.6004518422112963),
 np.int64(5): np.float64(0.8216709419733529),
 np.int64(6): np.float64(1.582070054621733)}

## Training

In [31]:
# Hyperparameters
start_lr = 0.0001
min_lr = 0.000001
max_lr = 0.001
warmup_epochs = 9
sustain_epochs = 0
factor = 0.96
epochs = 50

weight = {
    'cls': 1.0,
    'reg': 0.2,
}

# Initialize the model
model = FireRisk_Head(num_classes=7)
model = model.to(device)

criterion_cls = nn.CrossEntropyLoss()  # Cross-entropy loss for multi-class classification
criterion_reg = nn.MSELoss(reduction='none')  # Mean Squared Error loss for regression
optimizer = torch.optim.Adam(model.parameters(), lr=start_lr)

# lr_scheduler = CustomDecayLR(optimizer, epochs, warmup_epochs, sustain_epochs, factor, start_lr=start_lr, min_lr=min_lr, max_lr=max_lr)
# lr_scheduler = CustomPlateauLR(optimizer, epochs, warmup_epochs, sustain_epochs, factor, patience=2, start_lr=start_lr, min_lr=min_lr, max_lr=max_lr)
# lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=20, T_mult=2, eta_min=min_lr)
# early_stopping = EarlyStopping(patience=20, min_delta=0.0001, delta_metric='val_loss', start_epoch=5)

### Training loop

## History

# Evaluation

In [145]:
# Load precomputed latent representations and labels
test_data = torch.load('outputs/test_latents.pth', weights_only=True)
# test_latents, test_glcm, test_lbp, test_labels, test_labels_cnt, test_filenames = test_data['latents'], test_data['glcm'], test_data['lbp'], test_data['labels'], test_data['labels_cnt'], test_data['filenames']
test_latents, test_labels, test_labels_cnt, test_filenames = test_data['latents'], test_data['labels'], test_data['labels_cnt'], test_data['filenames']

# Min-max normalization
test_labels_cnt_norm = (test_labels_cnt - min_val) / (max_val - min_val)

# Create DataLoaders using the precomputed latents
# test_lat_dataset = LatentDataset(test_latents, test_glcm, test_lbp, test_labels, test_labels_cnt_norm, test_filenames)
test_lat_dataset = LatentDataset(test_latents, test_labels, test_labels_cnt_norm, test_filenames)
test_lat_loader = DataLoader(test_lat_dataset, batch_size=batch_size, shuffle=False)

In [146]:
# Initialize the FireRisk_Head model
model = FireRisk_Head(num_classes=7)
checkpoint = torch.load('models/02/model_epoch_40.pth')
model.load_state_dict(checkpoint)
model = model.to(device)

# # Initialize the combined model with the MAE encoder and FireRisk_Head weights
# full_model = FireRisk_Full(mae_model=mae_model, head_model=model, num_classes=7)
# full_model = full_model.to(device)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_956\1826315294.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('models/02/model_epoch_40.pth')


In [147]:
def evaluate(model, data_loader, criterion_cls, criterion_reg, device):
    model.eval()  # Set the model to evaluation mode
    running_loss = 0.0
    running_loss_reg = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    all_reg_preds = []
    all_reg_labels = []
    all_filenames = []

    with torch.no_grad():
        with tqdm(data_loader, unit="batch") as tepoch:
            for images, targets, target_reg, filenames in tepoch:
                images, targets, targets_reg = images.to(device), targets.to(device), target_reg.to(device)

                # Forward pass
                outputs_cls, outputs_reg = model(images)

                # Calculate loss
                targets_reg = targets_reg.to(torch.float).unsqueeze(1)
                mask_reg = targets_reg != 0
                
                if mask_reg.any():
                    loss_reg = criterion_reg(outputs_reg[mask_reg], targets_reg[mask_reg])
                else:
                    loss_reg = torch.tensor(0.0, device=device)
                    
                loss_cls = criterion_cls(outputs_cls, targets)

                running_loss += loss_cls.item()
                running_loss_reg += loss_reg.mean().item()

                # Calculate accuracy
                _, predicted = torch.max(outputs_cls, 1)
                total += targets.size(0)
                correct += (predicted == targets).sum().item()

                # Collect predictions and labels
                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(targets.cpu().numpy())
                # Save regression outputs and labels
                all_reg_preds.extend(outputs_reg.squeeze(1).cpu().numpy())
                all_reg_labels.extend(targets_reg.squeeze(1).cpu().numpy())
                all_filenames.extend(filenames)

                # Update the tqdm progress bar
                tepoch.set_postfix(loss=loss_cls.item(), accuracy=100 * correct / total)

    avg_loss = running_loss / len(data_loader)
    avg_loss_reg = running_loss_reg / len(data_loader)
    accuracy = 100 * correct / total
    
    return avg_loss, avg_loss_reg, accuracy, all_preds, all_labels, all_reg_preds, all_reg_labels, all_filenames

In [148]:
# Evaluate on test dataset
all_test_loss = []
all_test_loss_reg = []
all_test_accuracy = []
all_test_preds = []
all_test_labels = []
all_test_reg_preds = []
all_test_reg_labels = []
all_test_filenames = []

model_files = [
    "models/02/model_epoch_40.pth",
    "models/02/model_epoch_60.pth",
    "models/02/model_epoch_53.pth",
    "models/05/model_epoch_136.pth",
    "models/05/model_epoch_125.pth",
    "models/05/model_epoch_147.pth",
    "models/10/model_epoch_44.pth",
    "models/10/model_epoch_63.pth",
    "models/10/model_epoch_59.pth"
]

for i in range(len(model_files)):
    checkpoint = torch.load(model_files[i])
    model.load_state_dict(checkpoint)
    
    test_loss, test_loss_reg, test_accuracy, test_preds, test_labels, test_reg_preds, test_reg_labels, test_filenames = evaluate(model, test_lat_loader, criterion_cls, criterion_reg, device)
    all_test_loss.append(test_loss)
    all_test_loss_reg.append(test_loss_reg)
    all_test_accuracy.append(test_accuracy)
    all_test_preds.append(test_preds)
    all_test_labels.append(test_labels)
    all_test_reg_preds.append(test_reg_preds)
    all_test_reg_labels.append(test_reg_labels)
    all_test_filenames.append(test_filenames)
    print(f"Model {i+1}, Test Loss: {test_loss:.4f}, Test Loss Reg: {test_loss_reg:.4f}, Test Accuracy: {test_accuracy:.2f}%")

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_956\2348026881.py:24: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_files[i])
100%|███████████████

Model 1, Test Loss: 1.0366, Test Loss Reg: 0.0372, Test Accuracy: 61.13%


100%|███████████████████████████████████████████████| 1347/1347 [00:09<00:00, 137.28batch/s, accuracy=62.4, loss=0.573]


Model 2, Test Loss: 1.0150, Test Loss Reg: 0.0355, Test Accuracy: 62.43%


100%|███████████████████████████████████████████████| 1347/1347 [00:09<00:00, 136.92batch/s, accuracy=62.7, loss=0.155]


...

Model 7, Test Loss: 1.0829, Test Loss Reg: 0.0392, Test Accuracy: 61.04%


100%|███████████████████████████████████████████████| 1347/1347 [00:10<00:00, 129.62batch/s, accuracy=62.2, loss=0.325]


Model 8, Test Loss: 0.9992, Test Loss Reg: 0.0352, Test Accuracy: 62.15%


100%|███████████████████████████████████████████████| 1347/1347 [00:10<00:00, 130.33batch/s, accuracy=62.8, loss=0.236]

Model 9, Test Loss: 0.9769, Test Loss Reg: 0.0344, Test Accuracy: 62.76%


In [ ]:
# Plot Losses and Accuracies
plt.figure(figsize=(15, 4))

plt.subplot(1, 3, 1)
plt.plot(all_test_loss, label='Loss')
plt.xlabel('Model')
plt.ylabel('Loss')
plt.legend()
plt.title('Testing Loss')

plt.subplot(1, 3, 2)
plt.plot(all_test_loss_reg, label='Reg Loss')
plt.xlabel('Model')
plt.ylabel('Reg Loss')
plt.legend()
plt.title('Testing Reg Loss')

plt.subplot(1, 3, 3)
plt.plot(all_test_accuracy, label='Accuracy')
plt.xlabel('Model')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.title('Testing Accuracy')

plt.tight_layout()
plt.show()

In [157]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

all_test_mae = []
all_test_rmse = []
all_test_r2 = []

for i in range(len(model_files)):
    preds = np.array(all_test_reg_preds[i])
    targets = np.array(all_test_reg_labels[i])

    # Mask out targets == 0
    mask = targets != 0
    preds = preds[mask]
    targets = targets[mask]

    # Compute metrics
    mae = mean_absolute_error(targets, preds)
    rmse = mean_squared_error(targets, preds) ** 0.5
    r2 = r2_score(targets, preds)

    # Save
    all_test_mae.append(mae)
    all_test_rmse.append(rmse)
    all_test_r2.append(r2)

    # Print results
    print(f"Model {i+1} Metrics:")
    print(f"  MAE:  {mae:.4f}")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  R²:   {r2:.4f}")

Model 1 Metrics:
  MAE:  0.1791
  RMSE: 0.2255
  R²:   0.4477
Model 2 Metrics:
  MAE:  0.1657
  RMSE: 0.2207
  R²:   0.4709
Model 3 Metrics:
  MAE:  0.1598
  RMSE: 0.2175
  R²:   0.4862
Model 4 Metrics:
  MAE:  0.1720
  RMSE: 0.2266
  R²:   0.4419
Model 5 Metrics:
  MAE:  0.1658
  RMSE: 0.2202
  R²:   0.4730
Model 6 Metrics:
  MAE:  0.1568
  RMSE: 0.2120
  R²:   0.5118
Model 7 Metrics:
  MAE:  0.1734
  RMSE: 0.2319
  R²:   0.4155
Model 8 Metrics:
  MAE:  0.1681
  RMSE: 0.2196
  R²:   0.4759
Model 9 Metrics:
  MAE:  0.1653
  RMSE: 0.2172
  R²:   0.4875


## Results

### Save session results

In [48]:
test_results = pd.DataFrame({
    'Filename': test_filenames,
    'Label': test_labels,
    'Prediction': test_preds
})

os.makedirs('outputs', exist_ok=True)
test_results.to_csv('outputs/test_results.csv', index=False)

In [49]:
history_df = pd.DataFrame(history)
history_df.to_csv('outputs/history.csv', index=False)

In [50]:
all_test_results = pd.DataFrame({
    'all_test_loss': all_test_loss,
    'all_test_loss_reg': all_test_loss_reg,
    'all_test_accuracy': all_test_accuracy
})
all_test_results.to_csv('outputs/all_test_results.csv', index=False)